In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Reasoning Engine in Vertex AI - Multiple tools
@forusone (shins777@gmail.com)

This lab simplified the original colab [intro_reasoning_engine](https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/intro_reasoning_engine.ipynb) |Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

### Reasoning Engine in Vertex AI
This colab explains how to apply multiple tools for reasoning engine.

### Install Vertex AI SDK for Python

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 7.5 MB/s eta 0:00:00


In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Initial set up

In [3]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

### Tools definition

In [4]:
# Define Search tool

from vertexai.generative_models import grounding, Tool

grounded_search_tool = Tool.from_google_search_retrieval(
    grounding.GoogleSearchRetrieval()
)

In [6]:
# Get capital city
def get_capital_return(
    country:str, ):

  """Retrieves the capital city name of the given country"""

  from vertexai.generative_models import (
      GenerationConfig,
      GenerativeModel,
      HarmBlockThreshold,
      HarmCategory,
      Part,
  )

  model = GenerativeModel(MODEL)
  prompt = f"Answer the capital city of {country}"

  responses = model.generate_content(
        prompt,
  )

  return responses.text

In [7]:
# Get exchange rate
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

### Agent - Local unit test

In [13]:
from vertexai.preview.generative_models import ToolConfig

# Agent Define
agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    tools=[get_exchange_rate,    # Optional (Python function)
           get_capital_return, # Optional (Grounding Tool)
    ],
    # model_tool_kwargs={
    #     "tool_config": {
    #         "function_calling_config": {
    #             "mode": ToolConfig.FunctionCallingConfig.Mode.AUTO,
    #             #"allowed_function_names": ["search_arxiv", "get_exchange_rate"],
    #         },
    #     },
    # },

    agent_executor_kwargs={"return_intermediate_steps": False},
)

#### Test your agent locally


In [14]:
# Agent local unit test before deploying.
agent.query(input="what's the capital city of south korea")

{'input': "what's the capital city of south korea",
 'output': 'The capital city of South Korea is Seoul. \n'}

In [10]:
# Agent local unit test before deploying.
agent.query(input="What's the exchange rate from US dollars to EUR currency today?")

{'input': "What's the exchange rate from US dollars to EUR currency today?",
 'output': 'The exchange rate from US Dollar to EUR today is 0.94769 EUR per 1 USD. \n'}

### Deploy your agent on Vertex AI

In [15]:
# Crate Methods
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    display_name="multi tool agent",
    reasoning_engine_name="ai-agent", # does not support user-defined resource IDs at the moment
    gcs_dir_name = "ai-agent-1",
    description="This is a multi tool reasoning engine.",
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "requests",
    ],
    extra_packages = []
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket reasoning_7424
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning_7424/ai-agent-1/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning_7424/ai-agent-1/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning_7424/ai-agent-1/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/5120821474738831360/operations/8516843903708561408
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/721521243942/locations/us-central1/reasoningEngines/5120821474738831360
INFO:vertexai.reasoning_engines._reasonin

In [16]:
# query
remote_agent.query(
    input="What's the exchange rate from US dollars to China Yuan currency today?"
)

{'input': "What's the exchange rate from US dollars to China Yuan currency today?",
 'output': 'The exchange rate from US dollars to China Yuan is 7.2443 today. \n'}

In [17]:
# Properties of ReasoningEngine class.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine

#----------------------------------------
def get_reasoning_engine(display_name:str):
  try:
    for re in reasoning_engines.ReasoningEngine.list():
      if re.display_name == display_name:
        return reasoning_engines.ReasoningEngine(re.name)
      else:
        print("No such reasoning engine or Invalid display name.")

  except Exception as e:
    print(e)

#----------------------------------------
def get_re_list():
  """
  List reasoning engines.
  """

  try:
    if not reasoning_engines.ReasoningEngine.list():
      print("No reasoning engines")

    for idx, re in enumerate(reasoning_engines.ReasoningEngine.list()):
        print(f"Agent {idx}: \n\tDisplay Name [{re.display_name}] \n\tName [{re.name}] \n\tCreation Time [{re.create_time}] \n\tResource Name [{re.resource_name}]\n")

  except Exception as e:
    print(e)

#----------------------------------------

def del_re(name):
  """
  Delete a reasoning engines.
  @param name: The name of the reasoning engine.
  @type name: str
  """

  try:
    re = reasoning_engines.ReasoningEngine(name)
    re.delete()
    print(f"Deleted {name}")
  except Exception as e:
    print(e)


In [18]:
get_re_list()

Agent 0: 
	Display Name [multi tool agent] 
	Name [5120821474738831360] 
	Creation Time [2024-11-19 01:33:51.712140+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/5120821474738831360]

Agent 1: 
	Display Name [simple agent] 
	Name [6325534375060439040] 
	Creation Time [2024-11-18 13:34:12.086807+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/6325534375060439040]



In [19]:
re = get_reasoning_engine('multi tool agent')
re.query(
    input="what's the capital city of south korea?"
)

{'input': "what's the capital city of south korea?",
 'output': 'The capital city of South Korea is Seoul. \n'}

In [21]:
re = get_reasoning_engine('multi tool agent')
re.query(
    input="What's the exchange rate from US dollars to Korea won currency today?"
)

{'input': "What's the exchange rate from US dollars to Korea won currency today?",
 'output': 'The exchange rate from US dollars to Korea won today is 1 USD to 1397.36 KRW. \n'}